In [1]:
# Data Pipeline Notebook 1

# Download dataset from kaggle
# Load into DB Browser for SQLite
# Access Data using SQLAlchemy
# Clean Data
# Tokenize Data

# Data Pipeline Notebook 2

# Lemmatize
# PCA to achieve > 70% EVR
# Cluster Model
# Output top 10 closest books

In [17]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re
import pickle
import spacy
from spacy import displacy
import nltk
from nltk.util import ngrams
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
# https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=Books_rating.csv
engine = create_engine('sqlite:///book_review_DB.db')

In [21]:
df = pd.read_sql('SELECT * FROM books;', engine)

In [22]:
df.shape

(212405, 10)

In [23]:
df.drop([0], inplace = True)

In [25]:
df.head(1)

,Title,Description,Authors,ImageURL,PreviewURL,Publisher,PublishDate,InfoURL,Categories,Rating
1,Its Only Art If Its Well Hung!,None,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,None,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],None


In [26]:
drop_cols = ['Authors', 'ImageURL', 'Authors', 'PreviewURL', 'Publisher', 'PublishDate', 'InfoURL', 'Categories', 'Rating']

df_words = df.drop(drop_cols, axis = 1)

df_words.reset_index(drop = True, inplace = True)

In [28]:
df_words.dropna(inplace = True)

In [29]:
df_words.shape

(143962, 2)

In [30]:
df_words = df_words.head(100000)

In [31]:
df_words.shape

(100000, 2)

In [32]:
df_words.head()

,Title,Description
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...
8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...


In [33]:
unwanted_text = "[.,!?(){}\"•—\-\\:;@#$%^&*<>~=+/`_|1234567890]"

df_words['cleaned_1'] = df_words['Description'].map(lambda x: re.sub(unwanted_text, ' ', x))

In [34]:
df_words['cleaned_2'] = df_words['cleaned_1'].map(lambda x: x.lower())

In [36]:
df_words['Description'] = df_words['cleaned_2']

In [38]:
drop_cols = ['cleaned_1', 'cleaned_2']

df_words = df_words.drop(drop_cols, axis = 1)

In [41]:
df_words.head(1)

,Title,Description
1,Dr. Seuss: American Icon,philip nel takes a fascinating look into the k...


In [42]:
df_words.shape

(100000, 2)

In [45]:
len(df_words.Title.unique())

100000

In [46]:
with open('df_100k_desc', 'wb') as f:
    pickle.dump(df_words,f)